# Time series analysis with HyP3 and MintPy

This notebook walks through performing a time-series analysis for Mt. Edgecumbe, Alaska with On Demand InSAR products from the Alaska Satellite facility and MintPy. It follows the methodology of
[Grapenthin et. al., 2022]( https://doi.org/10.1029/2022GL099464) who used an InSAR time series analysis to identify volcanic activity in the area. In this notebook we'll:

1. Use the [ASF Search Python package](https://docs.asf.alaska.edu/asf_search/basics/) to:
   - Search ASF's catalog for Sentinel-1 SAR products covering the [Ridgecrest earthquake](https://earthquake.usgs.gov/storymap/index-ridgecrest.html)
   - Select a reference scene to generate a baseline stack
   - Select a [short baseline subset (SBAS)](https://docs.asf.alaska.edu/vertex/sbas/) of scene pairs for InSAR processing


2. Use the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/) to:
   - Request On Demand InSAR products from ASF HyP3
   - Download the InSAR products when they are done processing


3. Use [GDAL](https://gdal.org/api/index.html#python-api) and [MintPy](https://mintpy.readthedocs.io/en/latest/) to:
   - Prepare the InSAR products for MintPy
   - perform a time-series analysis with MintPy
   
---

**Note:** This notebook does assume you have some familiarity with InSAR processing with MintPy already, and is a minimal example without much context or explanations. If you're new to InSAR and MintPy, I suggest checking out:
* our [InSAR on Demand Story Map](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)


* [OpenSARlab's](https://opensarlab-docs.asf.alaska.edu/) highly detailed walkthrough of using HyP3 + MintPy via these notebooks:
  * [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
  * [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
  
    Note: While these notebooks make some assumptions you're working in OpenSARlab, you can run these 
    notebooks outside OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml).

## 0. Initial Setup

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:
```shell
conda create -n hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

conda activate hyp3-mintpy
jupyter notebook igarss_mtedgecumbe_ts_analysis.ipynb
```
Or, install these dependencies into your own environment:
```shell
conda install hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl
jupyter notebook igarss_mtedgecumbe_ts_analysis.ipynb
```

Then we'll start the data processing

First import the needed modules and set up our processing directories

In [ ]:
%matplotlib inline
import time
import numpy as np
from datetime import datetime
from pathlib import Path

import hyp3_sdk as sdk
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from mintpy.utils import readfile

## 1. Request On Demand InSAR products from ASF HyP3

Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3(prompt=True)

In [ ]:
project_name = 'edgecumbe_example'
reference = 'S1B_IW_SLC__1SDV_20180628T151540_20180628T151607_011575_015476_4673'
secondary = 'S1B_IW_SLC__1SDV_20190705T151547_20190705T151614_017000_01FFC4_D62F'

In [ ]:
import hyp3_sdk as sdk
jobs = sdk.Batch()
jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, 
                              include_dem=True, include_look_vectors=True, apply_water_mask=True)

In [ ]:
jobs = hyp3.watch(jobs)

In [ ]:
jobs = hyp3.find_jobs(name=project_name, status_code = 'SUCCEEDED')

In [ ]:
import hyp3_sdk as sdk
insar_products = jobs.download_files('.')
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]

In [ ]:
%matplotlib inline
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt

# TODO this is a placeholder scene
unwrapped_file = 'S1BB_20190822T151550_20191009T151551_VVP048_INT80_G_weF_2BF6/S1BB_20190822T151550_20191009T151551_VVP048_INT80_G_weF_2BF6_unw_phase.tif'
amp_file = 'S1BB_20190822T151550_20191009T151551_VVP048_INT80_G_weF_2BF6/S1BB_20190822T151550_20191009T151551_VVP048_INT80_G_weF_2BF6_amp.tif'

ds = gdal.Open(unwrapped_file)
unwrapped = np.ma.masked_equal(ds.GetRasterBand(1).ReadAsArray(), 0)
del ds

ds = gdal.Open(amp_file)
amp = np.ma.masked_equal(ds.GetRasterBand(1).ReadAsArray(), 0)
del ds

f, ax = plt.subplots(1,1, figsize=(6,6))
ax.imshow(unwrapped[1250:1575, 700:1000], alpha=0.75, cmap='jet')
ax.imshow(amp[1250:1575, 700:1000], alpha=0.33, cmap='Greys', norm=matplotlib.colors.LogNorm())
ax.set_title('Unwrapped Interferogram')

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()

## 3. Time-series Analysis with MintPy

### 3.1 Download the Staged Data

We are getting pair lists from the [supplementary information](https://zenodo.org/record/7151431) published by Grapenthin et. al.. Take a look at the `prep_data.py` script in the same directory as this notebook to see how we're doing this.

In [ ]:
import boto3
import os
import zipfile
from botocore import UNSIGNED
from botocore.client import Config

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
files = ['ascending_50.zip', 'ascending_79.zip', 'descending_174.zip', 'descending_174_nomask.zip']

for file in files:
    download_path = Path(file)
    s3_client.download_file('ffwilliams2-shenanigans', f'igarss_2023/{file}', download_path)
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(download_path.stem)
    os.unlink(download_path)

### 3.2 MintPy Background

In [ ]:
!smallbaselineApp.py --help

In [ ]:
!cat descending_174/mintpy_config.txt

### 3.3 Run MintPy Analysis

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection

from mintpy.cli import plot_coherence_matrix, tsview, plot_network, view
from mintpy.utils import readfile

In [ ]:
# Make sure you run MintPy for all four stacks
stacks = ['ascending_50', 'ascending_79', 'descending_174', 'descending_174_nomask']
stack = stacks[0]

#### 3.3.1 Check Network

In snowy areas like Mt. Edgecumbe, the use of a normal Small Baseline Subset (SBAS) network for Interferometric Synthetic Aperture Radar (InSAR) analysis may encounter certain limitations, particularly during the winter season. The presence of snow on the ground introduces significant decorrelation effects that hinder the accurate measurement of surface displacements. As a result, interferograms generated during winter periods tend to be unreliable and less informative. To overcome this challenge, it becomes necessary to exclude interferograms from the winter season. However, excluding winter interferograms leaves a data gap that can hamper the continuity of monitoring. To address this issue, an alternative approach is to create longer baseline interferograms, which span over a longer period of time and cover the missing winter period. By extending the baseline length, it becomes possible to capture a larger temporal span, allowing for a more comprehensive analysis of surface deformation, despite the limitations imposed by the snowy conditions. See the graph below to see the network that Grapenthin et. al. used.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection
from mintpy.utils.network import get_date12_list

pair_list = pd.DataFrame([pair.split('_') for pair in get_date12_list(f'{stack}/inputs/ifgramStack.h5')], columns=['date1', 'date2'])
pair_list['date1'] = pd.to_datetime(pair_list['date1'])
pair_list['date2'] = pd.to_datetime(pair_list['date2'])

# subplot 1 data
lines = []
bridges = []
for i, row in pair_list.iterrows():
    point1 = [mdates.date2num(row['date1']), i]
    point2 = [mdates.date2num(row['date2']), i]
    if point2[0] - point1[0] > 48:
        bridges.append([point1, point2])
    else:
        lines.append([point1, point2])

# subplot 2 data
date1_number = mdates.date2num(pair_list.date1)
date2_number = mdates.date2num(pair_list.date2)
dates = np.concatenate((date1_number, date2_number))
min_date = np.min(dates)
max_date = np.max(dates)

date_range = np.arange(np.min(dates), np.max(dates))
coverage = np.zeros(int(np.max(dates) - np.min(dates)))
for ifg_date1, ifg_date2 in zip(date1_number, date2_number):
    ifg_coverage = np.arange(ifg_date1 - min_date, ifg_date2 - min_date, dtype=int)
    coverage[ifg_coverage] += 1


f, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True)
line_segments = LineCollection(lines, array=range(len(lines)), linewidths=1, cmap='gist_rainbow', label='Interferograms')
bridge_segments = LineCollection(bridges, color='black', linestyle='dashed', linewidths=1, label='Bridging Interferograms')
ax1.add_collection(line_segments)
ax1.add_collection(bridge_segments)
ax1.set(
    ylabel='Interferogram Number',
    xlabel='Date',
    xlim=(mdates.date2num(np.min(pair_list['date1'])) - 10, mdates.date2num(np.max(pair_list['date2'])) + 10),
    ylim=(-1, pair_list.shape[0] + 1),
)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
ax1.legend(loc='upper left')

ax2.plot(date_range, coverage, color='black', linewidth=2)
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
ax2.set(xlabel='Date', ylabel='# of interferograms covering date', ylim=(0, np.max(coverage) + 1))

plt.tight_layout()

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --dostep modify_network

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mintpy.cli import plot_network

plot_network.main(f'{stack}/inputs/ifgramStack.h5 --figsize 12 3'.split())

#### 3.3.2 Check Reference Point

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --dostep reference_point

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mintpy.cli import view

cfg = readfile.read_template(f'{stack}/smallbaselineApp.cfg')
ref_la, ref_lo = [int(coord) for coord in cfg['mintpy.reference.lalo'].strip("[]").split(', ')]
view.main(f'{stack}/avgSpatialCoh.h5 --pts-lalo {ref_la} {ref_lo} --noverbose --figsize 8 8'.split())

#### 3.3.3 Estimate Velocity

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --start quick_overview

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mintpy.cli import tsview

cmd = f'{stack}/timeseries_ERA5_demErr.h5 --figsize 9 3 --noverbose'
tsview.main(cmd.split())

**Have you run MintPy for all four stacks!?**

## 4. Data Combination

**Make sure you've run MintPy for all four stacks**

### 4.1 Effect of Water Masking

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mintpy.utils import readfile

velocity_nomask, _ = readfile.read('descending_174_nomask/velocity.h5')
velocity_nomask = np.ma.masked_equal(velocity_nomask,0)

velocity_mask, _ = readfile.read('descending_174/velocity.h5')
velocity_mask = np.ma.masked_equal(velocity_mask,0)

difference = velocity_mask - velocity_nomask

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12, 6))
ax1.imshow(velocity_mask, cmap='jet', vmin=-0.04, vmax=0.07)
ax1.set(title='Water Masked')
ax2.imshow(velocity_nomask, cmap='jet', vmin=-0.04, vmax=0.07)
ax2.set(title='Not Water Masked')
diff_plot = ax3.imshow(difference, cmap='jet', norm=colors.CenteredNorm(halfrange=np.std(difference)))
ax3.set(title='Water Masked - Not Water Masked')

cax = ax3.inset_axes([1.04, 0.2, 0.05, 0.6])
f.colorbar(diff_plot, ax=ax3, cax=cax)

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()

### 4.2 Final Figure

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mintpy.utils import readfile

ascending_50, _ = readfile.read('ascending_50/velocity.h5')
ascending_50 = np.ma.masked_equal(ascending_50, 0)

ascending_79, _ = readfile.read('ascending_79/velocity.h5')
ascending_79 = np.ma.masked_equal(ascending_79, 0)

descending_174, _ = readfile.read('descending_174/velocity.h5')
descending_174 = np.ma.masked_equal(descending_174, 0)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12, 6))
ax1.imshow(ascending_79, cmap='jet', vmin=-0.04, vmax=0.07)
ax1.set(title='Ascending 79')
ax2.imshow(ascending_50, cmap='jet', vmin=-0.04, vmax=0.07)
ax2.set(title='Ascending 50')
cbar_plot = ax3.imshow(descending_174, cmap='jet', vmin=-0.04, vmax=0.07)
ax3.set(title='Descending 174')

cax = ax3.inset_axes([1.05, 0.25, 0.05, 0.5])
f.colorbar(cbar_plot, ax=ax3, cax=cax, orientation='vertical')

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()